# 📊 Data Visualizer — Workflow Guide

**Data Visualizer** follows the **Extract → Transform → Load (ETL)** flow to ingest **structured datasets** from public resources. It helps you **preview, clean, and visualize** data power by LLM, then (optionally) **import** it into a database (e.g., Snowflake, Supabase, PostgreSQL).  

## **Overview**

- **Data source:** UCI Machine Learning Repository, Data.gov, or your own CSV/ZIP uploads  
- **Tasks:** Preview data → Detect anomalies → Generate charts → AI summary  
- **Destinations (WIP):** Snowflake, Supabase, PostgreSQL
- **Data privacy:** Data Visualizer runs in your own Google runtime. If you prefer not to use Colab, download the notebook and run it locally.

---

## ✅ Quick Start
 **How to use this notebook**
1. Work **top-to-bottom**.
2. In each section labeled **Step #**, click **▶️ Run**.
3. If a cell has a form, fill it out **before** pressing ▶️ Run.
4. Review outputs in the **Results/Preview** panel beneath each step.

> **Tip:** If output keeps stacking, re-run the step—old output panels are cleared automatically.

---
<details>
  <summary>More information</summary>

## Part 1 — Pick a Dataset

Choose **one** of the following and press **▶️ Run**:
- **UCI (Search):** Type a keyword (e.g., *iris*, *heart*, *wine*) → **Search** → **Load Selected**  
- **Data.gov (Search):** Type a keyword (e.g., *medicare*, *CMS*) → **Search** → **Load Selected**  
- **Upload CSV/ZIP:** Click **Upload**, select files, then **Run Loader**  
- **Paste URL (CSV):** Paste a direct CSV link and **Run Loader**

**Output:** A `DATASETS` dictionary containing one or more pandas DataFrames.


## Part 2 — Explore & Analyze

Click **▶️ Run** on each analysis cell to:

- **Preview** the first rows and **summary statistics**  
- **Check data quality:** missing values, type mismatches, outliers  
- **Suggest charts:** quick ideas for histograms, scatter, and category plots  
- **AI summary (optional):** If you provide a Gemini API key, get a plain-English overview

> **Tip:** If an analysis cell mentions “Load a dataset first,” go back to **Part 1** and load data again.


## Part 3 — Load (WIP)

Select a target and press **▶️ Run**:
- **Destinations:** Snowflake, Supabase, PostgreSQL  
- **Credentials:** Provide URL, API key/token, or connection string  
- **Schema:** Tool can infer basic types; review before confirming

> **Security:** API keys are entered in a **password widget** and are **not stored** in your notebook file.

---

## 🧾 Notes

- The AI summary will only run if **Gemini** is configured in **Step 2**.  
- Some Data.gov resources block direct CSV fetch; the loader will attempt alternate methods automatically.
- A dict like: `{'uci_123_iris': DataFrame, 'datagov_hospital_ratings': DataFrame, ...}`
- Access with `DATASETS["name"]` to use a specific DataFrame in custom code.
<br><br>
---

## 🛠️ Troubleshooting

- **403 when downloading CSV from Data.gov:**  
  The loader retries with browser-style headers and CKAN API fallbacks.
- **No CSVs found:**  
  Check your folder name and that files end in `.csv` or `.data`.
- **Large files:**  
  Preview is limited (head + describe) to keep the notebook responsive, if colab ran out of RAM due to the large amount of data. Download the notebook and run it in your own local enviorment.
- **Additional data doesn't belong to the set:**  
  Delete any previously loaded data from the coblab upload folder (default foldername: **datasets**) if you see additional data loaded.

## 📧 Need help?

- Contact: siujeff [at] outlook [dot] com for questions and feedback.

---
</details>

In [ ]:
#@title ▶️ **Step 1 — Install libraries and functions** { display-mode: "form" }
#@markdown **What to do:** Click the **▶️ Run** button on the left of this cell.
#@markdown <br>**What this does:** Installs abd initialize required packages, libraries, and functions to run this notebook.
#@markdown _**Tips:** Click “Show code” if you want to see the details._

%%capture
!pip install -q ucimlrepo
!pip install -q -U supabase

from supabase import create_client, Client
import google.generativeai as genai
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output, Markdown
from IPython.utils import io
from google.colab import files, output
import io, struct, csv, re, os, subprocess, shutil, json, requests, time, traceback, threading, socket, datetime, zipfile, csv, pathlib, math
import base64, zlib
import google.generativeai as genai
import pandas as pd, numpy as np, requests, re, os, json, socket, datetime, csv
import matplotlib.pyplot as plt
from typing import List, Dict, Any, Tuple, Optional, Union
from urllib.parse import urlparse as _urlparse, quote
from ucimlrepo import fetch_ucirepo
from contextlib import contextmanager


# --- Functions ---
def clean_chunk(df):
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df = df.where(pd.notnull(df), None)
    return df.astype(object)

def is_json_serializable(record):
    try:
        json.dumps(record, allow_nan=False); return True
    except ValueError:
        return False

def convert_nan_to_none(d):
    return {k: (None if isinstance(v, float) and np.isnan(v) else v) for k, v in d.items()}

def insert_csv_to_supabase_in_chunks(table_name: str, supabase_url: str, supabase_key: str):
    """Reads {table_name}.csv and POSTs in chunks via REST."""
    csv_path = f"{table_name}.csv"
    api_url = f"{supabase_url}/rest/v1/{table_name}"
    headers = {
        "apikey": supabase_key,
        "Authorization": f"Bearer {supabase_key}",
        "Content-Type": "application/json",
        "Prefer": "resolution=merge-duplicates"
    }
    for chunk in pd.read_csv(csv_path, chunksize=1000):
        chunk = clean_chunk(chunk)
        payload = [convert_nan_to_none(r) for r in chunk.to_dict(orient="records")]
        payload = [r for r in payload if is_json_serializable(r)]
        if not payload:
            print("⚠️ Entire chunk was non-serializable, skipping."); continue
        r = requests.post(api_url, json=payload, headers=headers)
        r.raise_for_status()
        print(f"✅ Uploaded {len(payload)} records")

def get_column_name_from_csv():
    column_info, column_datatype_info = {}, {}
    csv_files = [f for f in os.listdir() if f.endswith(".csv")]
    for csv_file in csv_files:
        try:
            df = pd.read_csv(csv_file)
            column_info[csv_file[:-4]] = df.columns.tolist()
            column_datatype_info[csv_file] = df.dtypes.apply(lambda x: x.name).to_dict()
        except Exception as e:
            column_info[csv_file] = f"Error reading file: {e}"
    table_names = [csv_file[:-4] for csv_file in csv_files]
    return table_names, column_info, column_datatype_info

def create_dataframes_from_csv(folder_path="."):
    dataframes = {}
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            try:
                df = pd.read_csv(os.path.join(folder_path, filename))
                dataframes[filename[:-4]] = df
            except Exception as e:
                print(f"Error reading {filename}: {e}")
    return dataframes

def test_database_connection(supabase_url: str, supabase_key: str):
    try:
        socket.create_connection(("google.com", 80), timeout=5); print("Network connectivity: OK")
    except OSError as e:
        print(f"Network connectivity: FAIL - {e}")
    try:
        host = supabase_url.replace("https://", "").split("/")[0]
        socket.gethostbyname(host); print("DNS resolution: OK")
    except socket.gaierror as e:
        print(f"DNS resolution: FAIL - {e}")
    try:
        sb: Client = create_client(supabase_url, supabase_key)
        r = requests.get(f"{supabase_url}/rest/v1", timeout=8)
        print("Supabase project reachable:", r.status_code)
        return sb
    except Exception as e:
        print("Supabase not reachable:", e); return None

def create_supabase_table(sb: Client, table_name, df):
    """Print CREATE TABLE SQL (execute this in Supabase SQL editor)."""
    cols = []
    for col_name, col_type in df.dtypes.items():
        if pd.api.types.is_integer_dtype(col_type):  sql_type = "INT"
        elif pd.api.types.is_float_dtype(col_type): sql_type = "FLOAT8"
        elif pd.api.types.is_datetime64_any_dtype(col_type): sql_type = "TIMESTAMPTZ"
        else: sql_type = "TEXT"
        cols.append(f'"{col_name}" {sql_type}')
    create_table_sql = f'CREATE TABLE IF NOT EXISTS "{table_name}" ({", ".join(cols)});'
    print(f"Run this in SQL editor:\n{create_table_sql}")

def is_valid_sql_name(name):
    return bool(name) and len(name) <= 63 and re.match(r'^[a-zA-Z_][a-zA-Z0-9_]*$', name)

def fix_invalid_name(name):
    original = name
    name = re.sub(r'[^a-zA-Z0-9_]', '_', name)
    if not re.match(r'^[a-zA-Z_]', name): name = '_' + name
    if len(name) > 63: name = name[:63]
    if name != original: print(f"Invalid table name '{original}' fixed to '{name}'.")
    return name

def data_log(log_message):
    with open("etl_transformation.log", "a") as f:
        f.write(f"{datetime.datetime.now()}: {log_message}\n")

def extract_uci_dataset_id(url: str):
    try:
        path = urlparse(url).path
    except Exception:
        path = str(url)
    m = re.search(r'/dataset/(\d+)(?:/|$)', path)
    return int(m.group(1)) if m else None

def extract_ids_from_input(urls_csv: str):
    ids = []
    for u in [s.strip() for s in urls_csv.split(",") if s.strip()]:
        dsid = extract_uci_dataset_id(u)
        if dsid is not None: ids.append(dsid)
    return ids


In [ ]:
#@title ▶️ **Step 2 — Optional: Enable Gemini features** (Get your API key at https://aistudio.google.com/app/apikey) { display-mode: "form" }
#@markdown **What to do:** Click **▶️ Run**.
#@markdown <br>**What this does:** Lets you use Gemini for AI features. Your key is protected and will not be saved in the notebook.

# --- Widgets ---
gemini_api_key_input = widgets.Password(
    description='Gemini API Key:',
    value='',
    layout=widgets.Layout(width='700px'),
    style={'description_width': '95px'}
)

gemini_model_selector = widgets.Dropdown(
    options=[
        ('Gemini-2.5-pro (Solid reasoning)',          'gemini-2.5-pro'),
        ('Gemini-2.5-flash (Best price/performance)', 'gemini-2.5-flash'),
        ('Gemini-2.5-flash-lite (Fast & cheap)',      'gemini-2.5-flash-lite'),
        ('Gemini-2.0-flash (Good baseline)',          'gemini-2.0-flash'),
    ],
    value='gemini-2.5-flash',
    description='Gemini Model:',
    style={'description_width': '95px'},
    layout=widgets.Layout(width='700px')
)

btn_save_gemini = widgets.Button(description='Save & Test Gemini', button_style='primary')
out_gemini = widgets.Output()

# --- State (no key stored here) ---
_gemini_model = None

def get_selected_model_name():
    return gemini_model_selector.value

def save_and_test_gemini(_):
    global _gemini_model
    with out_gemini:
        clear_output()
        key = (gemini_api_key_input.value or '').strip()
        model_name = get_selected_model_name()
        if not key:
            print("⚠️ Please enter your Gemini API key.")
            return
        try:
            # Configure with the provided key (not stored globally)
            genai.configure(api_key=key)
            test_model = genai.GenerativeModel(model_name)
            resp = test_model.generate_content("Reply with the word: READY")
            if "READY" in (resp.text or "").upper():
                _gemini_model = test_model
                # Keep the key only in the environment, then clear references
                os.environ['GEMINI_API_KEY'] = key
                gemini_api_key_input.value = ''   # clears the password box
                del key                           # drop local reference
                print(f"✅ Gemini connected. Model: {model_name} ready to use.")
            else:
                print("⚠️ Connected, but unexpected response. Try another model.")
        except Exception as e:
            print("❌ Gemini test failed:", e)

btn_save_gemini.on_click(save_and_test_gemini)

def gemini_chat(prompt: str, system: str = None) -> str:
    """
    Calls Gemini using the in-memory model initialized above.
    Does not read or expose your API key.
    """
    if _gemini_model is None:
        raise RuntimeError("Gemini not initialized. Click 'Save & Test Gemini' first.")
    content = prompt if system is None else [{"role":"user","parts":[prompt]}]
    resp = _gemini_model.generate_content(content)
    return (resp.text or "").strip()

display(
    widgets.VBox([
        widgets.HTML("<b>Gemini Setup</b>"),
        gemini_api_key_input,
        gemini_model_selector,
        btn_save_gemini,
        out_gemini
    ])
)


In [ ]:
#@title ▶️ **Step 3 — Select a data source you would like to load (UCI / Data.gov / CSV)** { display-mode: "form" }
#@markdown **What to do:** Click **▶️ Run**.
#@markdown <br>**What this does:** Allow you to choose a dataset from a public data source.

try:
    from google.colab import files
except Exception:
    files = None

out = widgets.Output()

# --- UI: explicit modes (added data.gov search) ---
data_source_dropdown = widgets.Dropdown(
    options=[
        ('UCI Data Repository (Search)', 'uci_search'),
        ('Data.gov Public Dataset (Search)', 'datagov_search'),
        ('Dataset URL (csv)', 'uci_paste'),
        ('My own data (upload CSV/ZIP)', 'csv')
    ],
    value='uci_search',
    description='Data source:',
    layout=widgets.Layout(width='420px')
)

# common settings
target_folder_text = widgets.Text(value='datasets', description='Save to:', layout=widgets.Layout(width='300px'))
preview_rows_int   = widgets.BoundedIntText(value=5, min=1, max=50, step=1, description='Preview rows:')

save_csv_check     = widgets.Checkbox(value=True, description='Also save CSV to 📁')
settings_row       = widgets.HBox([target_folder_text, preview_rows_int, save_csv_check])

# ---------------- UCI search section ----------------
uci_search_text = widgets.Text(
    value='', placeholder='Search UCI data repository (e.g., medical, patient, diabetes)',
    description='Keywords:', layout=widgets.Layout(width='500px'),
    style={'description_width': '80px'}
)
btn_search_uci = widgets.Button(description='Search UCI')
uci_results_dropdown = widgets.Dropdown(options=[], description='Results:', layout=widgets.Layout(width='600px'),
                                        style={'description_width': '80px'})
btn_load_selected_uci = widgets.Button(description='Load Selected', button_style='primary')
uci_pick_section = widgets.VBox([
    widgets.HTML("<b>Enter anything in search below:</b>"),
    widgets.HBox([uci_search_text, btn_search_uci]),
    widgets.HBox([uci_results_dropdown, btn_load_selected_uci]),
])

# ---------------- data.gov search section ----------------
datagov_search_text = widgets.Text(
    value='', placeholder='Search dataset from Data.gov (e.g., medicare, HEDIS, CMS Star)',
    description='KeywordSearch:', layout=widgets.Layout(width='500px'),
    style={'description_width': '80px'}
)
btn_search_datagov = widgets.Button(description='Search data.gov')
datagov_results_dropdown = widgets.Dropdown(options=[], description='Results:', layout=widgets.Layout(width='600px'),
                                            style={'description_width': '80px'})
btn_load_selected_datagov = widgets.Button(description='Load Selected', button_style='primary')
datagov_pick_section = widgets.VBox([
    widgets.HTML("<b>Pick from data.gov (CSV resources)</b>"),
    widgets.HBox([datagov_search_text, btn_search_datagov]),
    widgets.HBox([datagov_results_dropdown, btn_load_selected_datagov]),
])

# ---------------- Paste section (IDs/URLs) ----------------
uci_urls_text = widgets.Text(
    value='',
    placeholder='(Experimental) Require Gemini API for non-CSV dataset extraction',
    description='IDs/URLs:',
    layout=widgets.Layout(width='700px')
)
uci_paste_section = widgets.VBox([
    widgets.HTML("<b>Extract dataset from URL</b>"),
    uci_urls_text,
    widgets.HTML('<span style="font-size:12px;color:#666">Examples: https://example.com/file.csv</span>')
])

# ---------------- Action row ----------------
upload_button = widgets.Button(description='Upload CSV/ZIP')
run_button    = widgets.Button(description='Run Loader', button_style='primary')
action_row    = widgets.HBox([upload_button, run_button])
_BROWSER_HEADERS = {
    # A modern browser UA
    "User-Agent": (
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/126.0 Safari/537.36"
    ),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.8",
    "Connection": "keep-alive",
}

_HTTP_HEADERS = {
    "User-Agent": _BROWSER_HEADERS["User-Agent"],
    "Accept": "text/csv,application/octet-stream;q=0.9,*/*;q=0.8",
}

# --- Helpers ---
def _is_http_url(s: str) -> bool:
    try:
        return _urlparse(str(s)).scheme in ("http", "https")
    except Exception:
        return False

def _download_browserish_csv(url: str) -> bytes:
    """
    For CKAN-style /dataset/.../resource/<uuid>/download/<file>.csv
    1) Warm up resource page to get cookies
    2) Download CSV with same session + referer
    Returns raw bytes.
    """
    u = _urlparse(url)
    path = u.path
    # Strip trailing /download/... to get the resource page as referer
    # e.g. /dataset/<dsid>/resource/<uuid>/download/file.csv -> /dataset/<dsid>/resource/<uuid>/
    referer_path = re.sub(r'/download/.*$', '/', path)
    referer = f"{u.scheme}://{u.netloc}{referer_path}"

    with requests.Session() as s:
        s.headers.update(_BROWSER_HEADERS)
        # Warm up (ignore failures; some pages block bots but still set cookies)
        try:
            s.get(referer, timeout=30, allow_redirects=True)
        except Exception:
            pass
        # Real download with referer
        headers = {**_HTTP_HEADERS, "Referer": referer}
        r = s.get(url, timeout=60, headers=headers, allow_redirects=True, stream=True)
        r.raise_for_status()
        return r.content

def _read_ckan_datastore(base: str, resource_id: str, page_size: int = 50000, max_pages: int = 20) -> pd.DataFrame:
    frames, offset = [], 0
    api = f"{base}/api/3/action/datastore_search"
    with requests.Session() as s:
        s.headers.update({**_HTTP_HEADERS, "X-Requested-With": "XMLHttpRequest"})
        for _ in range(max_pages):
            r = s.get(api, params={"resource_id": resource_id, "limit": page_size, "offset": offset}, timeout=60)
            r.raise_for_status()
            res = r.json().get("result", {})
            recs = res.get("records", [])
            if not recs:
                break
            frames.append(pd.DataFrame.from_records(recs))
            offset += len(recs)
            if len(recs) < page_size:
                break
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

def _read_csv_http_with_fallback(url: str) -> pd.DataFrame:
    # 1) Try browser-ish direct download
    try:
        raw = _download_browserish_csv(url)
        delim = _sniff_delimiter(raw[:64 * 1024])
        return pd.read_csv(io.BytesIO(raw), delimiter=delim)
    except requests.HTTPError as e:
        # 2) If 403, try CKAN datastore (if we have a resource UUID)
        status = getattr(e.response, "status_code", None)
        if status == 403:
            m = re.search(r"/resource/([0-9a-f-]{36})/", url, flags=re.I)
            if m:
                base = f"{_urlparse(url).scheme}://{_urlparse(url).netloc}"
                df = _read_ckan_datastore(base, m.group(1))
                if not df.empty:
                    return df
        # else rethrow
        raise

def _safe_name(name: str) -> str:
    name = name.strip().replace(" ", "_")
    return re.sub(r"[^A-Za-z0-9._-]+", "_", name)

def _sniff_delimiter(sample_bytes: bytes) -> str:
    try:
        sample = sample_bytes.decode("utf-8", errors="ignore")
        dialect = csv.Sniffer().sniff(sample, delimiters=[",",";","|","\t"])
        return dialect.delimiter
    except Exception:
        return ","

def _read_csv_smart(path_or_url: str) -> pd.DataFrame:
    # Local file?
    if not _is_http_url(path_or_url):
        try:
            return pd.read_csv(path_or_url)
        except Exception:
            with open(path_or_url, "rb") as f:
                sample = f.read(64 * 1024)
            delim = _sniff_delimiter(sample)
            return pd.read_csv(path_or_url, delimiter=delim)

    # HTTP(S)
    try:
        # First try a simple read (fast path); pandas may succeed for many hosts
        return pd.read_csv(path_or_url)
    except Exception:
        # Use robust path with headers + CKAN fallback
        return _read_csv_http_with_fallback(path_or_url)


def _collect_csv_like(folder: str) -> List[str]:
    outp = []
    for root, _, files_ in os.walk(folder):
        for f in files_:
            if f.lower().endswith((".csv", ".data")):
                outp.append(os.path.join(root, f))
    return outp

def _ensure_folder(folder: str):
    os.makedirs(folder, exist_ok=True)

def extract_uci_dataset_id(url_or_id: str):
    s = str(url_or_id).strip()
    if s.isdigit():
        return int(s)
    path = urlparse(s).path
    m = re.search(r'/dataset/(\d+)(?:/|$)', path)
    return int(m.group(1)) if m else None

def _load_by_id(dsid: int, folder: str, preview_rows: int, save_csv: bool):
    ds = fetch_ucirepo(id=int(dsid))
    X, y = ds.data.features, ds.data.targets
    df = pd.concat([X, y], axis=1) if y is not None else X
    key = re.sub(r'[^A-Za-z0-9_]+','_', f"uci_{ds.metadata.id}_{ds.metadata.name}").lower()
    globals().setdefault("DATASETS", {})[key] = df
    if save_csv:
        _ensure_folder(folder)
        df.to_csv(os.path.join(folder, f"{key}.csv"), index=False)
    out.clear_output(wait=True)
    with out:
        print(f"✅ Loaded UCI {ds.metadata.id}: {ds.metadata.name} — shape {df.shape}")
        display(df.head(preview_rows))
        print(f"🧰 Access with: DATASETS['{key}']")

def _load_csv_from_url(url: str, folder: str, preview_rows: int, save_csv: bool, key_prefix="ext"):
    df = _read_csv_smart(url)
    name = os.path.basename(urlparse(url).path) or "data.csv"
    name = re.sub(r'\.csv$', '', name, flags=re.I)
    key = re.sub(r'[^A-Za-z0-9_]+','_', f"{key_prefix}_{name}").lower().strip('_')
    globals().setdefault("DATASETS", {})[key] = df
    if save_csv:
        _ensure_folder(folder)
        df.to_csv(os.path.join(folder, f"{key}.csv"), index=False)
    out.clear_output(wait=True)
    with out:
        print(f"✅ Loaded CSV from URL — shape {df.shape}")
        display(df.head(preview_rows))
        print(f"🧰 Access with: DATASETS['{key}']")

# -------- UCI search (unchanged) --------
def _search_uci_datasets(query: str, limit: int = 25):
    if not query.strip():
        return []
    url = f"https://archive.ics.uci.edu/datasets?search={quote(query.strip())}"
    try:
        html = requests.get(url, timeout=30, headers=_HTTP_HEADERS).text
    except Exception:
        return []
    matches = re.findall(r'href="/dataset/(\d+)/([^"]+)"', html)
    outl, seen = [], set()
    for id_str, slug in matches:
        if id_str in seen:
            continue
        seen.add(id_str)
        pretty = re.sub(r'[\+\-_]+', ' ', slug).title()
        label = f"{pretty} (id {id_str})"
        outl.append((label, int(id_str)))
        if len(outl) >= limit:
            break
    return outl

# -------- data.gov search --------
def _search_datagov_csv_resources(query: str, limit: int = 50):
    """
    Uses CKAN search on catalog.data.gov and returns CSV resources.
    Output: list of tuples (label, url)
    """
    if not query.strip():
        return []
    api = "https://catalog.data.gov/api/3/action/package_search"
    try:
        r = requests.get(api, params={"q": query, "rows": limit}, timeout=30, headers=_HTTP_HEADERS)
        r.raise_for_status()
        out, seen = [], set()
        for pkg in r.json().get("result", {}).get("results", []):
            pkg_title = pkg.get("title") or pkg.get("name") or "dataset"
            for res in pkg.get("resources", []):
                fmt = (res.get("format") or "").lower()
                mtype = (res.get("mimetype") or "").lower()
                url = res.get("url") or ""
                if not url:
                    continue
                is_csv = fmt == "csv" or "text/csv" in mtype or url.lower().endswith(".csv")
                if not is_csv:
                    continue
                label = f"{pkg_title} — {res.get('name') or 'CSV'}"
                if url in seen:
                    continue
                seen.add(url)
                out.append((label, url))
        return out
    except Exception:
        return []

# --- UI show/hide logic ---
def _show(w, on=True):
    w.layout.display = '' if on else 'none'

def _update_ui(change=None):
    mode = data_source_dropdown.value
    _show(uci_pick_section,       mode == 'uci_search')
    _show(datagov_pick_section,   mode == 'datagov_search')
    _show(uci_paste_section,      mode == 'uci_paste')
    if mode in ('uci_search', 'datagov_search'):
        upload_button.layout.display = 'none'
        run_button.layout.display    = 'none'   # use respective "Load Selected" button
    elif mode == 'uci_paste':
        upload_button.layout.display = 'none'
        run_button.layout.display    = ''       # Run for pasted IDs/URLs
    else:  # csv
        upload_button.layout.display = ''       # upload files
        run_button.layout.display    = ''       # Run to parse/preview

data_source_dropdown.observe(_update_ui, names='value')

# --- Callbacks ---
def on_upload_clicked(_):
    with out:
        clear_output()
        if files is None:
            print("Uploads not available here."); return
        print("Select one or more CSV/ZIP files…")
        uploaded = files.upload()
        folder = target_folder_text.value.strip() or "datasets"
        _ensure_folder(folder)
        saved = []
        for name, data in uploaded.items():
            path = os.path.join(folder, _safe_name(name))
            with open(path, "wb") as f:
                f.write(data)
            saved.append(path)
        for p in saved:
            if p.lower().endswith(".zip"):
                with zipfile.ZipFile(p, "r") as zf:
                    zf.extractall(folder)
                os.remove(p)
        print("✅ Upload complete. Files saved to:", os.path.abspath(folder))

@contextmanager
def busy(msg="Loading data…", buttons=()):
    # disable buttons to prevent double-clicks
    for b in buttons:
        b.disabled = True

    # clear only the Output panel, then show the message
    out.clear_output(wait=True)
    with out:
        print(f"⏳ {msg}")
    try:
        yield
    finally:
        for b in buttons:
            b.disabled = False

def on_run_clicked(_):
    # reset only the Output panel
    out.clear_output(wait=True)
    with out:
        print("⏳ Loading…")

    folder = target_folder_text.value.strip() or "datasets"
    _ensure_folder(folder)

    mode = data_source_dropdown.value
    if mode == 'uci_paste':
        items = [u.strip() for u in uci_urls_text.value.split(",") if u.strip()]
        with out:
            if not items:
                print("⚠️ Please provide at least one UCI dataset ID or CSV URL.")
                return
        for u in items:
            dsid = extract_uci_dataset_id(u)
            try:
                if dsid is not None:
                    _load_by_id(dsid, folder, preview_rows_int.value, save_csv_check.value)
                else:
                    _load_csv_from_url(u, folder, preview_rows_int.value, save_csv_check.value, key_prefix="url")
            except Exception as e:
                with out:
                    print(f"❌ Failed to load '{u}': {e}")
        return

    # csv mode
    csv_like = _collect_csv_like(folder)
    with out:
        if not csv_like:
            print("⚠️ No CSV/DATA files found in:", os.path.abspath(folder))
            return

        dataframes: Dict[str, pd.DataFrame] = {}
        print(f"✅ Discovered {len(csv_like)} CSV-like file(s):")
        for p in sorted(csv_like):
            print(" •", os.path.relpath(p, folder))
            try:
                df = _read_csv_smart(p)
                key = pathlib.Path(p).stem
                base, i = key, 2
                while key in dataframes:
                    key = f"{base}_{i}"; i += 1
                dataframes[key] = df
            except Exception as e:
                print(f"   ↳ ❌ Failed to read: {e}")

        print("\n📋 Preview:")
        for name, df in dataframes.items():
            print(f"\n{name} — shape: {df.shape}")
            display(df.head(preview_rows_int.value))

        globals()["DATASETS"] = dataframes
        globals()["DATASETS_FOLDER"] = os.path.abspath(folder)
        print(f"\n🧰 Use DATASETS['name'] to access a DataFrame.")
        print(f"📂 Files are under: {os.path.abspath(folder)}")


def _on_search_uci(_):
    out.clear_output(wait=True)
    with out:
        print("Searching UCI…")
    results = _search_uci_datasets(uci_search_text.value)
    uci_results_dropdown.options = results
    out.clear_output(wait=True)
    with out:
        print("No matches. Try another keyword." if not results
              else f"Found {len(results)} match(es). Pick one, then click 'Load Selected'.")

def _on_load_selected_uci(_):
    dsid = uci_results_dropdown.value
    if dsid is None:
        out.clear_output(wait=True)
        with out: print("Pick a result first.");
        return
    with busy("Loading UCI dataset…", buttons=(btn_search_uci, btn_load_selected_uci)):
        _load_by_id(dsid, target_folder_text.value or "datasets", preview_rows_int.value, save_csv_check.value)


def _on_search_datagov(_):
    out.clear_output(wait=True)
    with out:
        print("Searching data.gov… (CSV resources)")
    results = _search_datagov_csv_resources(datagov_search_text.value)
    datagov_results_dropdown.options = results
    out.clear_output(wait=True)
    with out:
        print("No matches or no CSV resources. Try another keyword." if not results
              else f"Found {len(results)} CSV resource(s). Pick one, then click 'Load Selected'.")

def _on_load_selected_datagov(_):
    url = datagov_results_dropdown.value
    if not url:
        out.clear_output(wait=True)
        with out: print("Pick a result first.");
        return
    with busy("Loading data.gov CSV…", buttons=(btn_search_datagov, btn_load_selected_datagov)):
        _load_csv_from_url(url, target_folder_text.value or "datasets", preview_rows_int.value, save_csv_check.value, key_prefix="datagov")


# bind AFTER definitions
upload_button.on_click(on_upload_clicked)
run_button.on_click(on_run_clicked)
btn_search_uci.on_click(_on_search_uci)
btn_load_selected_uci.on_click(_on_load_selected_uci)
btn_search_datagov.on_click(_on_search_datagov)
btn_load_selected_datagov.on_click(_on_load_selected_datagov)

# initial visibility + render
def _render():
    _update_ui()
    display(widgets.VBox([
        data_source_dropdown,
        uci_pick_section,
        datagov_pick_section,   # NEW
        uci_paste_section,
        settings_row,
        action_row,
        out
    ]))

_render()


In [ ]:
#@title ▶️ **Step 4 — Provide a summary of the data from your previous selection** (require Gemini API) { display-mode: "form" }
#@markdown **What to do:** Click **▶️ Run**.
#@markdown <br>**What this does:** Generate a summary of the data using Gemini API.

PROMPT_BLOB = "eJyVk8GO1DAMhl_Fx5mqrZZdlsNcEBKCAwIhBrS3RW7jacOkcRUn0-2eeAfekCfBactKnIBbEv-xf39O3jIb-MYNGGrZWN9B7K3AQCLYEZjd_df7fXOoquoTOzrAkbzlAAYjAnp0s8QaXiVjybca9uyrSG3vbYuufsPO8aQZSQhCciRAD9hGNx8qOEYMEWZOAQKNbobJxj5rVw046zVhUTzbw12PEdTVYm0t3XCKL4uigs89ebC-dcmQqq839dl6A3zSyInDgNGyB8MwEfR4Ib2p7o3qbzZ9iz4Hs2S1sVVaanxMEQjbHoTaJVNPuLDSpY0CPPnFbqnloGFnyt99AgpIr_G6gi8K4T2Gs8ly9m6u4QPrjRgYIj3EUumBDoGgcdyeZdlHbBTbsuw52Ef2Ed3Kcjk0VlqHdqAgWuId0aiOIHlDAW6vrmDiYHLkTjHRRQ8VuE7XL22gn7WgS4OX7Dyzzz0LRdAZr2OLFAY5qI_QkVocrEju_IIuZQs6Bme1eAlqQzTN0KBDfQsAU8AxJxly7qLIXBT7juquLnW_ZiyK_XGDaiiidXL468ivf37_8Rya5JwaHVkJKuQ0DBjm-j9eQAWvc944j7mRMz2xKLduYWdPmdG-fMIi9pFquFEDL_40UP_jU8pXb2EM-jryF9n-kH4MBX6xktBphaDtdB3J4jmyimSiUP8CmohFxw=="

def get_caesar_command(msg):
    s = msg.strip()
    s += "=" * (-len(s) % 4)                 # fix missing padding
    raw = base64.urlsafe_b64decode(s)        # URL-safe decode
    return zlib.decompress(raw).decode("utf-8")

def keep_after_marker(text: str, marker: str = "---") -> str:
    i = text.lower().find(marker.lower())
    if i != -1:
        return text[i:].lstrip()
    parts = re.split(r'(?m)^\s*---+\s*$', text, maxsplit=1)
    return parts[1].lstrip() if len(parts) > 1 else text

def build_llm_dataset_report(datasets: dict,
                             max_rows: int = 5,
                             max_cols_listed: int = 40,
                             max_chars: int = 60000) -> str:

    if not isinstance(datasets, dict) or not datasets:
        return "No datasets loaded."

    parts = []
    for name, df in datasets.items():
        # info()
        info_buf = io.StringIO()
        try:
            df.info(buf=info_buf)
            info_txt = info_buf.getvalue()
        except Exception:
            info_txt = "(info unavailable)"

        # describe() with fallback
        try:
            desc = df.describe(include="all").T
        except Exception:
            desc = df.select_dtypes(include="number").describe().T

        # robust datetime summary
        dt_cols = [c for c in df.columns if pd.api.types.is_datetime64_any_dtype(df[c])]
        dt_txt = ""
        if dt_cols:
            dt_num = pd.DataFrame({
                c: pd.to_datetime(df[c], utc=True, errors="coerce").astype("int64") / 1e9
                for c in dt_cols
            })
            dt_txt = "\n-- datetime (secs since epoch) describe --\n" + dt_num.describe().T.to_string()

        # limit columns listing
        columns_list = list(map(str, df.columns[:max_cols_listed]))
        if df.shape[1] > max_cols_listed:
            columns_list.append("...")

        head_txt = df.head(max_rows).to_string(index=False)

        part = (
            f"\n### DATASET: {name}\n"
            f"shape: {df.shape}\n"
            f"columns: {', '.join(columns_list)}\n"
            f"\n-- head({max_rows}) --\n{head_txt}\n"
            f"\n-- describe --\n{desc.to_string()}\n"
            f"{dt_txt}\n"
            f"\n-- info --\n{info_txt}\n"
        )
        parts.append(part)

    report = "\n" + ("\n" + "="*70 + "\n").join(parts)
    if len(report) > max_chars:
        report = report[:max_chars] + "\n...[truncated]..."
    return report

# Get Gemini report
if 'DATASETS' in globals() and DATASETS:
    if '_gemini_model' in globals() and _gemini_model is not None:
        report_text = build_llm_dataset_report(DATASETS)
        prompt = keep_after_marker(get_caesar_command(PROMPT_BLOB))
        reply = gemini_chat(prompt+report_text)
        display(HTML("""
          <style>
            .output .markdown, .output .markdown * {
              font-size: 24px !important;
              line-height: 1.6 !important;
            }
          </style>
        """))
        display(Markdown(reply))

    else:
        print("⚠️ Gemini API not initialized. Please run Step 2 to enable Gemini features.")
else:
    print("⚠️ No datasets loaded. Please run Step 3 to load a dataset.")

In [ ]:
#@title ▶️ **Step 5 — Provide a Plots & Charts using LLM** (require Gemini API) { display-mode: "form" }
#@markdown **What to do:** Click **▶️ Run**.
#@markdown <br>**What this does:** Plots & Charts using LLM.

PROMPT_BLOB = "eJydUstuE0EQvO9XlOaUSHaCxENihXKAQwRCNnJOESJW727bHjw7beaBvUSR8g_whfkSetcOIhFw4DbTU13VNV3nIg0-S4WGa2msXyKtbETLMdKS0Rxdza-Oq7IYj8fFjFMOHtPJ-0u8u5hOcOQFLYV1I1s_gl42QSIfYyHOyfYXWaxX3FJZXBeAqVcUUjTlR70Ap6d4I9knyAKEmhIvJdiaHGpxufUD6NqkbsOmNHUPNSOz0_Or38Bn5mZ0T3ceJG-4QUUBAz4-5ND6fLnH_JFpZFaZ_8F_kaheK7-X0JKz3_SorjcSkhUfUXUY2B-Kxn3TvEf-l-pr2WHjJJXwuWV9xtd4_13dI4Oy-4tE19cO_UqubZ-Km6KYZcdRN4ypdx0CLziwr_mwgag7pATe2ZhOFPRhAGhRAY5itAul770jUVhyAkUMZrBdsUfMNlHluG99enf7_QX2AdDc7Pra237vgb9kjkm_cq8JDY0ETaPmoMr6c4p3ds0wT8bP1Mjz8UszQkzWOeTIsMNkk6m2ty17HaS7u_0xRFTU00lRXEoGBVatyN5KQEOJQCrQqS3MmJrBUl9eBGr5EFqFHParMvvJy-InkkoNYw=="

# ----------------- Utilities -----------------
def _latest_dataset(dsets: dict) -> tuple[str, pd.DataFrame]:
    if not isinstance(dsets, dict) or not dsets:
        raise RuntimeError("No DATASETS loaded.")
    key = next(reversed(dsets))
    return key, dsets[key].copy()

def _infer_col_types(df: pd.DataFrame) -> dict:
    out = {}
    for c in df.columns:
        dt = str(df[c].dtype)
        if pd.api.types.is_numeric_dtype(df[c]):
            out[c] = "numeric"
        elif pd.api.types.is_bool_dtype(df[c]):
            out[c] = "boolean"
        elif pd.api.types.is_datetime64_any_dtype(df[c]):
            out[c] = "datetime"
        else:
            # treat few-unique objects as categorical
            nunique = df[c].nunique(dropna=True)
            out[c] = "categorical" if nunique <= max(30, int(len(df)*0.2)) else "text"
    return out

def _example_values(df, col, k=6):
    vals = df[col].dropna().astype(str).unique()[:k]
    return list(map(str, vals))

def _build_schema_for_llm(df: pd.DataFrame, limit_rows=6):
    # small preview + column inventory to guide Gemini
    col_types = _infer_col_types(df)
    cols = []
    for c in df.columns:
        cols.append({
            "name": c,
            "inferred_type": col_types[c],
            "examples": _example_values(df, c)
        })
    sample = df.head(limit_rows).to_dict(orient="records")
    return {"columns": cols, "sample_rows": sample}

def _extract_json_block(text: str):
    # try to pull the first {...} block
    m = re.search(r'\{[\s\S]*\}', text)
    return m.group(0) if m else None

# ------------- Chart rendering helpers -------------
def _try_order_ordinal(series: pd.Series):
    """
    Attempt to order categories like '0-4','5-9', '10-14', or single numbers as strings.
    Returns (ordered_categories, converter_func_or_None).
    """
    vals = series.dropna().astype(str).unique().tolist()
    # detect ranges "a-b"
    bounds = []
    for v in vals:
        m = re.match(r'^\s*(\d+)\s*[-–]\s*(\d+)\s*$', v)
        if m:
            lo, hi = int(m.group(1)), int(m.group(2))
            bounds.append((v, (lo+hi)/2))
        else:
            m2 = re.match(r'^\s*(\d+)\s*$', v)
            if m2:
                bounds.append((v, float(m2.group(1))))
            else:
                return None, None  # give up
    bounds.sort(key=lambda x: x[1])
    ordered = [b[0] for b in bounds]
    return ordered, dict(bounds)  # map label->numeric midpoint

def _count_plot(df, x):
    vc = df[x].dropna().astype(str).value_counts()
    if vc.empty:
        print(f"(skip) No data for count plot of {x}"); return
    ax = vc.plot(kind='bar')
    ax.set_title(f"Count of {x}")
    ax.set_xlabel(x); ax.set_ylabel("count")
    plt.show()

def _bar_grouped_counts(df, x, hue, stacked=False, normalize=False):
    if x not in df or hue not in df:
        print(f"(skip) Missing columns for grouped bar: {x}, {hue}"); return
    tbl = pd.crosstab(df[x].astype(str), df[hue].astype(str), normalize='index' if normalize else False)
    ax = tbl.plot(kind='bar', stacked=stacked)
    ax.set_title(f"{'Proportion' if normalize else 'Counts'} of {x} by {hue}")
    ax.set_xlabel(x); ax.set_ylabel("proportion" if normalize else "count")
    plt.legend(title=hue, bbox_to_anchor=(1.04,1), loc="upper left")
    plt.tight_layout()
    plt.show()

def _box_by_category(df, x, y):
    if x not in df or y not in df:
        print(f"(skip) Missing columns for boxplot: {x}, {y}"); return
    s_x = df[x].astype(str)
    y_num = pd.to_numeric(df[y], errors='coerce')
    if y_num.notna().sum() < 2:
        print(f"(skip) Not enough numeric data for {y}"); return

    # try ordinal ordering on x
    ordered, mapping = _try_order_ordinal(s_x)
    if ordered:
        s_x = pd.Categorical(s_x, categories=ordered, ordered=True)
    groups = [y_num[s_x == lvl].dropna().values for lvl in pd.Series(s_x).cat.categories] if hasattr(s_x, "cat") else \
             [y_num[s_x == lvl].dropna().values for lvl in sorted(s_x.unique())]
    labels = (list(pd.Series(s_x).cat.categories) if hasattr(s_x, "cat") else sorted(s_x.unique()))
    if len(labels) < 2:
        print(f"(skip) Only one category in {x}"); return

    plt.figure()
    plt.boxplot(groups, labels=labels, showfliers=False)
    plt.title(f"{y} by {x}")
    plt.xlabel(x); plt.ylabel(y)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# ------------- End-to-end: ask Gemini -> render -------------
def request_chart_plan_from_gemini(df: pd.DataFrame, dataset_observation: str) -> dict:
    """
    Ask Gemini for a JSON chart plan. Expects your `gemini_chat()` wrapper to be available.
    """
    schema = _build_schema_for_llm(df)
    prompt = (keep_after_marker(get_caesar_command(PROMPT_BLOB))+
              f"SCHEMA:\n{json.dumps(schema, ensure_ascii=False)}\n\n"
              f"Dataset observation: \n{dataset_observation}"
             )
    raw = gemini_chat(prompt)  # uses your existing wrapper
    blob = _extract_json_block(raw) or raw
    try:
        plan = json.loads(blob)
        assert isinstance(plan, dict) and "charts" in plan
        return plan
    except Exception as e:
        print("Could not parse Gemini response as JSON. Raw response:\n", raw)
        raise

def render_chart_plan(df: pd.DataFrame, plan: dict):
    for spec in plan.get("charts", []):
        t = spec.get("type")
        try:
            if t == "count":
                _count_plot(df, spec["x"])
            elif t == "bar_grouped":
                _bar_grouped_counts(df, spec["x"], spec["hue"], stacked=False, normalize=False)
            elif t == "stacked_prop":
                _bar_grouped_counts(df, spec["x"], spec["hue"], stacked=True, normalize=True)
            elif t == "box":
                _box_by_category(df, spec["x"], spec["y"])
            else:
                print(f"(skip) Unknown chart type: {t}")
        except Exception as e:
            print(f"(skip) Failed to render {t}: {e}")

# Use dataset summary from step 4
if 'DATASETS' in globals() and DATASETS:
    if '_gemini_model' in globals() and _gemini_model is not None:
        key, df0 = _latest_dataset(DATASETS)

        # Get the dataset observation from the previous Gemini reply stored in globals
        # Check if `reply` is defined and not empty
        dataset_observation = globals().get('reply', '') # Get the 'reply' variable, default to empty string if not found

        if not dataset_observation:
             print("⚠️ Dataset observation from Step 4 is not available or is empty. Please run Step 4 first.")
        else:
            # 1) Ask Gemini for a plan
            plan = request_chart_plan_from_gemini(df0, dataset_observation)
            print("Based on the dataset provided, below are a few plots that can help visualize your data: ", plan)  # so you can see what it decided

            # 2) Render it
            render_chart_plan(df0, plan)
    else:
        print("⚠️ Gemini API not initialized. Please run Step 2 to enable Gemini features.")
else:
    print("⚠️ No datasets loaded. Please run Step 3 to load a dataset.")

In [ ]:
#@title ▶️ **Step 6 — Gather statistic from the dataset tables** { display-mode: "form" }
#@markdown **What to do:** Click **▶️ Run**.
#@markdown <br>**What this does:** (use 📊 button to the right of the dataframe for plots with SeaBorn)

# Display the dataset and provide option for statistical analysis
print("--- Loaded Datasets Summary ---")
if 'DATASETS' in globals() and DATASETS:
    for name, df in DATASETS.items():
        print(f"Dataset: '{name}'")
        print(f"  Number of records: {df.shape[0]}")
        print(f"  Number of columns: {df.shape[1]}")
        display(df)
else:
    print("No datasets have been loaded yet.")